## Setup

In [0]:
from src.steam.bronze import (
    pull_games_steamspy,
    pull_player_count_steam,
    pull_global_achievements_steam
)

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS bronze;
CREATE DATABASE IF NOT EXISTS silver;
CREATE DATABASE IF NOT EXISTS gold;

## Get All Steam Apps

The Steam `getapplist` endpoint is deprecated and no longer reliable for obtaining a complete list of app IDs and game names.  
SteamSpy provides a more stable alternative via the `request=all` endpoint, which returns all Steam apps in paginated form.

**Important:** SteamSpy enforces a **60-second delay between page requests**, making this ingestion step a long-running process.

If runtime is a concern and perfect completeness is not required, you may choose to use `getapplist` instead.


In [0]:
pull_games_steamspy(spark)

## API Usage Warning

Steam enforces a limit of 100,000 API calls per 24 hours per key/IP.

Both of the following ingestion steps can consume ~87,000 calls each:

- `pull_player_count_steam()`
- `pull_achievements_steam()`

SteamSpy does include a current player count and a range of owners for each game in their `request=all`. <br>
However, after reviewing these numbers it is clear that they are extremely innacurate/unreliable and often just return null. <br>
Because of this I have opted to use Steam's `GetNumberOfCurrentPlayers` from their official API.

### Recommended execution order

1. Run `pull_player_count_steam()`
2. Wait **24 hours** (to reset Steam API limits)
3. Run `pull_achievements_steam()`

In [0]:
pull_player_count_steam(spark)

In [0]:
pull_global_achievements_steam(spark)